In [106]:
import sys
sys.path.append('..')

import numpy as np
import socnet as sn

In [107]:
sn.graph_width = 500
sn.graph_height = 500
sn.node_size = 10
sn.edge_width = 1
sn.edge_color = (192, 192, 192)
sn.node_label_position = 'top center'

In [108]:
total_nodes = 200
percentage_women = 0.25
percentage_man = 1 - percentage_women

g = sn.generate_empty_graph(total_nodes)
g = g.to_directed()

In [109]:
from random import randint

for n in range(int(total_nodes * percentage_man)):
    g.node[n]['gender'] = 'male'
    g.node[n]['ability'] = randint(0, 10)
    g.node[n]['color'] = (255,255,0)
    g.node[n]['label'] = n
    
for n in range(int(total_nodes * percentage_man), total_nodes):
    g.node[n]['gender'] = 'female'
    g.node[n]['ability'] = randint(0, 10)
    g.node[n]['color'] = (255,0,255)
    g.node[n]['label'] = n


In [110]:
for n in g.nodes():
    for i in g.nodes():
        if n != i:
            ability_n = g.node[n]['ability']
            ability_i = g.node[i]['ability']
            if g.node[n]['gender'] == 'male' and g.node[i]['gender'] == 'female':
                ability_i = max(ability_i - 3,0)
            if ability_i >= ability_n and randint(0,50) == 50:
                g.add_edge(n,i)

In [111]:
sn.reset_edge_colors(g)
sn.show_graph(g, nlab=True)

In [112]:
from scipy.stats.stats import pearsonr
from scipy.optimize import minimize

def equals(a, b):
    return abs(a - b) < 0.000000001


def calculate_pagerank(g):
    length = g.number_of_nodes()

    k = 10
    scale = 0.8
    residue = (1 - scale) / length

    R = sn.build_matrix(g)

    for n in g.nodes:
        total = np.sum(R[n,])

        if equals(total, 0):
            R[n, n] = 1
        else:
            R[n,] /= total

    R = scale * R + residue

    Rt = R.transpose()

    rank = 1 / length

    r = np.full((length, 1), rank)

    for _ in range(k):
        r = Rt.dot(r)

    for n in g.nodes:
        g.nodes[n]['pagerank'] = r[n, 0]

In [113]:
calculate_pagerank(g)

In [114]:
result = []
avg_pagerank_male = 0
avg_pagerank_female = 0
for n in g.nodes():
    result.append({'index': n, 'gender': g.node[n]['gender'], 'pagerank': g.node[n]['pagerank']})
    if (g.node[n]['gender'] == 'male'):
        avg_pagerank_male += g.node[n]['pagerank']
    else:
        avg_pagerank_female += g.node[n]['pagerank']

avg_pagerank_male /= (total_nodes * percentage_man)
avg_pagerank_female /= (total_nodes * percentage_women)

print('avg_male', avg_pagerank_male)
print('avg_female', avg_pagerank_female)
# result = sorted(result, key=itemgetter('pagerank'), reverse=True)

# for n in result:
#     print(n)

avg_male 0.0057585100824
avg_female 0.00272446975279
